# Kubernetes

In [ ]:
# WEBDAV at http://10.15.136.41:30080/
# visualization
# local 
# kubernetesTool


In [ ]:
print(getPods()['items'][10].keys())
getPods()['items'][10]['metadata']['name']
getPods()['items'][9]['status'].keys()


In [ ]:
# download e.g. all crts of needed peers to local dir (e.g. for invoke,commit)
def getCrtName(org='', domain='kubernetes.research.dev.seeburger.de'):
    return str(org + '-' + domain + '-ca.crt')
    
def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='upload'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


def downloadCrtsToPeer(orgs, execute_org='schalke3'):
    for org in orgs:
        executePeerCmd(peers[execute_org].pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name=getCrtName(org))))     
   

In [ ]:
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

In [ ]:
# hyperledger peer

def deletePeer(peername):   
    output  = executeKubectlCmd(['delete','configmap','hl-fabric-peer-' + peername])
    #output  = executeKubectlCmd(['delete','-f','/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer.yaml'])
    output  = executeKubectlCmd(['delete','deployment',peername])
    output  = executeKubectlCmd(['delete', 'service', peername])
    return output
            
def addEnvVariable(pod_name, key,value):
    try:  
        cmd = "echo \'export '" + key + '=' + value + "'\' >> /root/.bashrc"
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', 'scray-peer-cli','--', '/bin/bash','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
       
def addBashEnvToPeer(peer):        
    pod_name    = peer.pod_name
    listen_port = peer.listen_port
    executePeerCmd(pod_name,"rm -f /root/.bashrc")
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli touch /root/.bashrc
    addEnvVariable(pod_name, 'ORDERER_CA', '/tmp/tlsca.example.com-cert.pem')
    addEnvVariable(pod_name, 'ORDERER_HOSTNAME', 'orderer.example.com')
    addEnvVariable(pod_name, 'CHANNEL_NAME', 'mychannel')
    #addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', pod_name + ':' + listen_port)
    #addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', 'peer0.kubernetes.research.dev.seeburger.de:' + listen_port)
    addEnvVariable(pod_name, 'CORE_PEER_ADDRESS', 'kubernetes.research.dev.seeburger.de:' + listen_port)
    addEnvVariable(pod_name, 'CORE_PEER_LOCALMSPID', peer.name + 'MSP')
    addEnvVariable(pod_name, 'CORE_PEER_MSPCONFIGPATH', '/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp')
    addEnvVariable(pod_name, 'SHARED_FS_HOST', '10.15.136.41:30080')
    addEnvVariable(pod_name, 'SHARED_FS_USER', 'scray')
    addEnvVariable(pod_name, 'SHARED_FS_PW', 'scray')        
  
def addBashEnvToPeers(peerlist):
    for name in peerlist:
        if name in peers:
            peer = peers[name]
            addBashEnvToPeer(peer)
            #installAndAproveChaincode(peer)

##############
           
# actions

#!/home/jovyan/work/usr/bin/kubectl exec -it tanja1-65b7889d68-ghxpj -c scray-peer-cli -- /bin/bash -c "curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o /tmp/org3_update_in_envelope.pb"
def sign(pod_name):
    #executePeerCmd(pod_name,"curl --user $SHARED_FS_USER:$SHARED_FS_PW http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem > /tmp/tlsca.example.com-cert.pem"
    executePeerCmd(pod_name,"curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o /tmp/org3_update_in_envelope.pb")
    executePeerCmd(pod_name,"peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb")
    executePeerCmd(pod_name,"curl --user scray:scray -T /tmp/org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb")
       

def installAndAproveChaincode(peer,cc_host='10.14.128.38',orderer_host='10.15.128.97'):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer_host]
    executePeerCmd(peer.pod_name,toCmd(strlist)) 
                    
#########################################################################            
      

In [ ]:
cc_host='10.14.128.38'
orderer_host='10.15.128.97'
peer=peers['oly2']
strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer_host]
executePeerCmd2(peer.pod_name,toCmd(strlist)) 

In [ ]:
# Cerificates
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID

#https://stackoverflow.com/questions/16899247/how-can-i-decode-a-ssl-certificate-using-python

def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    cmd = ['curl', '--user',user + ':' + pwd, 
           'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt']
    res = executeLocalCmd(cmd)
    if 'CERTIFICATE' in str(res):
        return res
       
def getOrgCrt(org=''):    
    pem_data=getCrt(org)
    if pem_data:
        return x509.load_pem_x509_certificate(pem_data, default_backend())


In [ ]:
# get config of blockchain and store it at local directory of peer
#peer channel fetch config  $WORK_LOCATION/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA 
#configtxlator proto_decode --input $WORK_LOCATION/config_block.pb --type common.Block | jq .data.data[0].payload.data.config > $WORK_LOCATION/config.json

def getCurrentConfig(peer, channel_name='mychannel', location='/tmp'):
    pod_name = peer.pod_name
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_path='ca', remote_name='tlsca.example.com-cert.pem')))               
    cmd = 'peer channel fetch config ' + location + '/config_block.pb -o orderer.example.com:7050 -c ' +  channel_name + ' --tls --cafile /tmp/tlsca.example.com-cert.pem'
    executePeerCmd(pod_name,cmd)
    cmd = 'configtxlator proto_decode --input ' + location + '/config_block.pb --type common.Block > ' + location + '/config_block.json'
    executePeerCmd(pod_name,cmd)
    cmd = 'jq .data.data[0].payload.data.config ' + location + '/config_block.json > ' + location + '/config.json'
    executePeerCmd(pod_name,cmd)
    cmd = 'cat ' + location + '/config.json'
    return json.loads(executePeerCmd(pod_name,cmd))
    
def getConsortium(config):
    dkeys = config.get('channel_group').get('groups').get('Application').get('groups').keys()
    return dkeys    
    
def getLocalOrgList(data):    
    local_orgs=[]
    for name in data:
        if name[:-3] in peers:
            local_orgs.append(name[:-3])    
    return local_orgs

def getBlockInfo(peer=None,channel='mychannel'):
    strlist = ['peer', 'channel','-c', channel, 'getinfo']
    return executePeerCmd(peer.pod_name,toCmd(strlist))
    #return json.loads(executePeerCmd(peer.pod_name,toCmd(strlist))[17:])

def getChannelList(peer=None):
    strlist = ['peer', 'channel', 'list']
    return executePeerCmd2(peer.pod_name,toCmd(strlist))


In [ ]:
# update local_orgs

config = getCurrentConfig(peers['oly0'])
#getChannelList(peers['oly0'])

consortium = getConsortium(config)
local_orgs = getLocalOrgList(consortium)

In [ ]:
import json
#len(kubernetesTool.getPods())
#kubeconfig = kubernetesTool.getConfigmap()
from kubernetesTool import executeKubectlCmd
#executeKubectlCmd(['get','configmap', '-o', 'json'])
kubeconfig = json.loads(executeKubectlCmd(['get','configmap', '-o', 'json']))
len(kubeconfig['items'])

In [ ]:
pods = json.loads(executeKubectlCmd(['get', 'pods', '-o', 'json']))  


In [ ]:
pods['items'][1]['metadata']['name']
len(pods['items'])


In [ ]:
#pods['items'][1]['metadata']
deployments = json.loads(executeKubectlCmd(['get', 'deployments', '-o', 'json'])) 

In [ ]:
#deployments['items'][1]['metadata'].keys()
#deployments['items'][14]['metadata']['name']

deploymentNames=[]
for item in deployments['items']:
    deploymentNames.append(item['metadata']['name'])

In [ ]:
def getLocalOrgListFromDeploymentNames(consortium,deploymentNames):    
    local_orgs=[]
    for name in consortium:
        if name[:-3] in deploymentNames:
            local_orgs.append(name[:-3])    
    return local_orgs

local_orgs = getLocalOrgListFromDeploymentNames(consortium, deploymentNames)

In [ ]:
#from kubernetesTool import executeLocalCmd
for name in local_orgs:
    peer = kubernetespeer(name=name)
    peers[name] = peer

In [ ]:
#peers, local_orgs

# Kubernetes Version (Current)

In [43]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort

## addPeer

In [44]:
def addPeer(peername):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/ && /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'))
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            
#createPeers('oly')
#deletePeers('ga')

In [45]:
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=' + peer_host_name,
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=CORE_PEER_ADDRESS=peer0.' + peer_host_name + ':' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + peer_host_name + ':' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)  

In [46]:
def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [47]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [48]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]


#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## check status peers

In [107]:
from kubernetesTool import executeKubectlCmd
_res = executeKubectlCmd(['get', 'pods', '-o', 'json'], decode='json')

for item in _res['items']:
    labels=item['metadata']['labels']
    #status=item['status']['initContainerStatuses'][0]
    #print(labels,status['state'],status['restartCount'],status['name'],status['ready'])
    print(labels,item['status']['phase'])
    #print(labels)

{'app': 'hl-fabric-data-share', 'pod-template-hash': '6849487f76'} Running
{'app': 'invoice-chaincode-external', 'pod-template-hash': '847cd44954'} Running
{'app': 'lalya-0', 'pod-template-hash': '7c887cffb8'} Running
{'app': 'lalya-1', 'pod-template-hash': '7c76dff8c'} Running
{'app': 'lalya-2', 'pod-template-hash': '5f8b45dbc4'} Running
{'app': 'orderer-org1-scray-org', 'pod-template-hash': '9dbd9fc68'} Running
{'app': 'peer48', 'pod-template-hash': '57445d5f7b'} Running


In [74]:
_res['items'][3]['metadata']['labels']
_res['items'][0]['status']['phase']

'Running'

## create peers

In [49]:
peers = {}

In [78]:
peers

{'lalya-0': <__main__.kubernetespeer at 0x7f7dac574640>,
 'lalya-1': <__main__.kubernetespeer at 0x7f7dac536700>,
 'lalya-2': <__main__.kubernetespeer at 0x7f7dac536730>}

In [81]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')

In [51]:
createPeers('lalya',start=0, end=3)

In [ ]:
orderer.__dict__

In [ ]:
kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/inform_existing_nodes.sh $ORDERER_IP $CHANNEL_NAME $PEER_NAME $SHARED_FS_HOST $EXT_PEER_IP $PEER_HOST_NAME

In [ ]:
EER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
 ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
 ORDERER_PORT=7050
 PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
 kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST $EXT_PEER_IP

In [99]:
#orderer_ip=get_orderer_ip('orderer-org1-scray-org') 
#orderer_hostname='orderer.example.com'
#orderer_port='7050'

orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
#orderer_port=orderer.orderer_intern
orderer_port= orderer.listen_port

channel_name='super43'
shared_fs_host='10.14.128.38:30080'
    
def endorse_and_join(peer):    
    res1 = endorse_peer(peer=peer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
    res2  = joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host)
    return res1,res2

for key, value in peers.items():
    endorse_and_join(value)

lalya-0
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super43 lalya-0 10.14.128.38:30080 10.15.136.41 lalya-0.kubernetes.research.dev.seeburger.de


2021-10-07 07:55:57.873 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:55:57.899 UTC [cli.common] readBlock -> INFO 002 Received block: 3
2021-10-07 07:55:57.900 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 3
2021-10-07 07:55:57.919 UTC [cli.common] readBlock -> INFO 004 Received block: 3
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   237  100   237    0     0  10304      0 --:--:-- --:--:-- --:--:-- 10304
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847    0     0  100   847      0  44578 --:--:-- --:--:-- --:--:-- 44578
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
10

/mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41
/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
2021-10-07 07:55:59.552 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:55:59.563 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-10-07 07:55:59.631 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:55:59.819 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


lalya-1
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super43 lalya-1 10.14.128.38:30080 10.15.136.41 lalya-1.kubernetes.research.dev.seeburger.de


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
2021-10-07 07:56:01.762 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:01.771 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-10-07 07:56:01.852 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:01.971 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


lalya-2
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super43 lalya-2 10.14.128.38:30080 10.15.136.41 lalya-2.kubernetes.research.dev.seeburger.de


2021-10-07 07:56:02.337 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:02.349 UTC [cli.common] readBlock -> INFO 002 Received block: 3
2021-10-07 07:56:02.349 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 3
2021-10-07 07:56:02.356 UTC [cli.common] readBlock -> INFO 004 Received block: 3
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   237  100   237    0     0  59250      0 --:--:-- --:--:-- --:--:-- 59250
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847    0     0  100   847      0   165k --:--:-- --:--:-- --:--:--  165k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
10

/mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41
/home/jovyan/work/usr/bin/kubectl exec -t lalya-2-5f8b45dbc4-52tt7 -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   137k      0 --:--:-- --:--:-- --:--:--  137k
2021-10-07 07:56:03.977 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:03.992 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-10-07 07:56:04.051 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:04.165 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5f97c57d44-j5mbs -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super40 kalya-0 10.14.128.38:30080 10.15.136.41 kalya-0.kubernetes.research.dev.seeburger.de

In [ ]:
type(peers)

## install chaincode

In [94]:
channel_name
orderer.__dict__

{'name': 'orderer-org1-scray-org',
 'pod_name': 'orderer-org1-scray-org-9dbd9fc68-5lrgq',
 'listen_port': '30081',
 'gossip_port': '',
 'host': 'orderer.example.com',
 'orderer_intern': 7050}

In [100]:
#cc_host chaincode host
def installAndAproveChaincode(peer,orderer,channel_name,cc_host='10.14.128.38'):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
     
for key, peer in peers.items():
    installAndAproveChaincode(peer,orderer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super43


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   6081      0 --:--:-- --:--:-- --:--:--  6081
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-10-07 07:56:32.992 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [663ae957435c3928f4673b317fd44928a73dc53ca00e49f49c235a7357439396] committed with status (VALID) at peer0.lalya-0.kubernetes.research.dev.seeburger.de:30700


/home/jovyan/work/usr/bin/kubectl exec -t lalya-1-7c76dff8c-pzx78 -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super43


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   6521      0 --:--:-- --:--:-- --:--:--  6521
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-10-07 07:56:35.983 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [42d14bc33c7343a67aff03cf2c57cee6ba6713585ecf92ce7d7f23cd30e55e13] committed with status (VALID) at peer0.lalya-1.kubernetes.research.dev.seeburger.de:32294


/home/jovyan/work/usr/bin/kubectl exec -t lalya-2-5f8b45dbc4-52tt7 -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super43


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   7142      0 --:--:-- --:--:-- --:--:--  7142
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-10-07 07:56:38.818 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [37653ed57eeeae1fa4b83ce55ce4f8bc8cd2b5e6ccfb471d239df6307752b6d7] committed with status (VALID) at peer0.lalya-2.kubernetes.research.dev.seeburger.de:30851


In [101]:
def commitChaincode(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

peer = list(peers.values())[0]
commitChaincode(peer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc_commit.sh super43


2021-10-07 07:57:09.677 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [782c14433b3559febd81e46c921fe1fbcb24a5dda8f240bbbceb223bf52a017f] committed with status (VALID) at peer0.lalya-0.kubernetes.research.dev.seeburger.de:30700


"Installed chaincode\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee, Label: basic_1.0\nChaincode definition for chaincode 'basic', version '1.0', sequence '1' on channel 'super43' approval status by org:\nAdminOrgMSP: false\nlalya-0MSP: true\nlalya-1MSP: true\nlalya-2MSP: true\n"

In [105]:
peer.name, channel_name

('lalya-0', 'super43')

In [106]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/cc-basic-interaction.sh  super43

Error: endorsement failure during invoke. response: status:500 message:"error in simulation: failed to execute transaction 42d0040afd7a8c8a129f481f503bb8828b87eceed4a967b663d8c0f4e3a017fe: could not launch chaincode basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee: connection to basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee failed: error cannot create connection for basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee: error creating grpc connection to asset-transfer-basic.org1.example.com:30999: failed to create new connection: connection error: desc = \"transport: error while dialing: dial tcp 10.14.128.38:30999: connect: connection refused\"" 
Error: endorsement failure during query. response: status:500 message:"error in simulation: failed to execute transaction 6baa63b319b0947a9c9d3f3c748efc7ac1681b5c6330dd441a0258232e2a59e6: could not launch chaincode basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd

In [108]:
#Call init method in chaincode
def chaincode_init(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc-basic-interaction.sh', channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

chaincode_init(peer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc-basic-interaction.sh super43


2021-10-07 09:08:07.889 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


'[{"Key":"invoice1","Record":{"ID":"invoice1","InvoiceOwner":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE","ProductByer":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE"}}]\n'

In [ ]:
CHANNEL_NAME=mychannel
#ORDERER_NAME=orderer.example.com
IP_CC_SERVICE=10.14.128.38         # Host where the chaincode is running
#PEER_POD=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
#ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')
#ORDERER_LISTEN_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
#ORDERER_HOST=orderer.example.com
EXT_PEER_IP=10.14.128.38

ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=7050
ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')

kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $ORDERER_IP $ORDERER_HOST $ORDERER_PORT $CHANNEL_NAME 

In [ ]:
#peers.keys()
#[elem for elem in peers.values()]
list(peers.values())[0].__dict__

## invoice usecase

In [123]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername):
    return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
    #return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE,C=DEdx509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

/mnt/conf/peer/add-invoice.sh super43 046 x509::CN=User1@lalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-1.kubernetes.research.dev.seeburger.de,O=lalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE
/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/add-invoice.sh super43 046 x509::CN=User1@lalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-1.kubernetes.research.dev.seeburger.de,O=lalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE


2021-10-07 10:24:09.284 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


''

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [122]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/get-my-invoices.sh super43


'[{"Key":"invoice1","Record":{"ID":"invoice1","InvoiceOwner":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE","ProductByer":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE"}}]\n'

In [112]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

/mnt/conf/peer/transfer_invoice.sh super43 045 x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE
/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/transfer_invoice.sh super43 045 x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE


/bin/sh: /mnt/conf/peer/transfer_invoice.sh: Permission denied
command terminated with exit code 126


"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-t', 'lalya-0-7c887cffb8-fpw7h', '-c', 'scray-peer-cli', '--', '/bin/sh', '-c', '/mnt/conf/peer/transfer_invoice.sh super43 045 x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE']' returned non-zero exit status 126."

In [114]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

'x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

In [119]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


'lalya-0-7c887cffb8-fpw7h'

In [121]:
peers[pernames[2]].pod_name

'lalya-2-5f8b45dbc4-52tt7'

## add channel

In [82]:
orderer.pod_name

'orderer-org1-scray-org-9dbd9fc68-5lrgq'

In [97]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super43 orderer.example.com 30081

2021-10-07 07:52:53.181 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2021-10-07 07:52:53.204 UTC [common.tools.configtxgen.localconfig] Load -> INFO 002 Loaded configuration: /mnt/conf/orderer/configtx.yaml
2021-10-07 07:52:53.205 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003 Generating new channel configtx
2021-10-07 07:52:53.207 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 004 Writing new channel tx
2021-10-07 07:52:53.274 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:52:53.341 UTC [cli.common] readBlock -> INFO 002 Expect block, but got status: &{NOT_FOUND}
2021-10-07 07:52:53.345 UTC [channelCmd] InitCmdFactory -> INFO 003 Endorser and orderer connections initialized
2021-10-07 07:52:53.547 UTC [cli.common] readBlock -> INFO 004 Expect block, but got status: &{SERVICE_UNAVAILABLE}
2021-10-07 07:52:53.553 UTC [channelCmd] InitCmdFactory -> INFO 005 Endorser and or

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]


# Multiple version

In [ ]:
# init
peers = {}

In [ ]:
# create peer
peername='olya4'
addPeer(peername)
peers[peername].__dict__

In [ ]:
# initialize blockchain from known peer
import kubernetesTool
peer = kubernetespeer(name=peername)
#kubernetesTool.getPod('oly0')
peers[peername] = peer

In [ ]:
peername='test27'
#executeLocalCmd('cd /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/ && /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/configure-deployment.sh -n ' + peername)
#!ls /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/
#executeLocalCmd(['/bin/cd','/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/'])
#executeLocalCmd('which cd')
cmd = "cd"
#subprocess.check_output(cmd)
cmd ='!cd /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/ && pwd'
cmd=['/bin/bash','&&','/usr/bin/ls','/home']
cmd=['/bin/bash','-c','cd /']
subprocess.check_output(cmd)
#subprocess.run(cmd, stdout = subprocess.PIPE, stderr=subprocess.PIPE)
#!which cd

In [ ]:
def getPeer0Address(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def getPeerBaseAddress(peer):
    addr='kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

In [ ]:
import kubernetesTool

peer = peers['test4']
cmd= ['get', 'pods', peer.pod_name, '-o', 'json']
#peer_containers_text.value=str(len(json.loads(executeKubectlCmd(cmd).decode('ascii'))['spec']['containers']))
kubernetesTool.executeKubectlCmd(cmd)

In [ ]:
import os
import json
import subprocess
import pandas as pd

import containerWidgets
from containerWidgets import addHeader,addVisText,addVisCheckbox   
from kubernetesTool import executeKubectlCmd

def setPeerWidgetValues(peer):
    pod_name_text.value = peer.pod_name
    peer_addr_text.value = getPeerAddress(peer)
    cmd= ['get', 'pods', peer.pod_name, '-o', 'json']
    peer_containers_text.value=str(len(json.loads(executeKubectlCmd(cmd))['spec']['containers']))
          
    cert = getOrgCrt(org=peer.name)
    if cert:
        issuer = cert.issuer
        c_o_text.value=issuer.get_attributes_for_oid(NameOID.ORGANIZATION_NAME)[0].value
        c_cn_text.value=issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value
        c_sn_text.value=str(cert.serial_number)
        c_l_text.value=issuer.get_attributes_for_oid(NameOID.LOCALITY_NAME)[0].value
        c_s_text.value=issuer.get_attributes_for_oid(NameOID.STATE_OR_PROVINCE_NAME)[0].value
        c_c_text.value=issuer.get_attributes_for_oid(NameOID.COUNTRY_NAME)[0].value
        c_nvb_text.value=cert.not_valid_before.strftime("%Y-%m-%d %H:%M:%S")
        c_nva_text.value=cert.not_valid_after.strftime("%Y-%m-%d %H:%M:%S")
    else:
        c_o_text.value=''
    

def handle_submit(text):
    global peername
    peer = addPeer(text.value)
    setPeerWidgetValues(peer)
    
layout2 = {'width': '600px'}   
addHeader(text='peer',bold=True)
peertext      = addVisText(key='organization',layout=layout2)
pod_name_text = addVisText(key='pod name',disabled=True,layout=layout2)
peer_containers_text = addVisText(key='containers',disabled=True,layout=layout2)
peer_addr_text = addVisText(key='address',disabled=True,layout=layout2)
peertext.on_submit(handle_submit)    

addHeader(text='certificate',bold=True)
c_o_text = addVisText(key='organization',disabled=True,layout=layout2)
c_cn_text = addVisText(key='common name',disabled=True,layout=layout2)
c_sn_text =addVisText(key='serial number',disabled=True,layout=layout2)
c_l_text =addVisText(key='locality',disabled=True,layout=layout2)
c_s_text =addVisText(key='state',disabled=True,layout=layout2)
c_c_text =addVisText(key='country',disabled=True,layout=layout2)
c_nvb_text =addVisText(key='from',disabled=True,layout=layout2)
c_nva_text =addVisText(key='until',disabled=True,layout=layout2)

Step2: Sign by majority of old peers

In [ ]:
# download current organizations (consortium) from sharepoint
downloadToLocal(local_path='/tmp', local_name='org_msp.json' ,remote_path='upload')
with open('/tmp/org_msp.json', 'r') as fp:
    data = json.load(fp)
       
local_orgs = getLocalOrgList(data)    

In [ ]:
for name in local_orgs:
    sign(peers[name].pod_name)

Step 3

In [ ]:
peer = peers['oly2']
addBashEnvToPeer(peer)

peer.pod_name

In [ ]:
#pod_name   = peers['sally5'].pod_name
orderer_ip = '10.15.128.97'   
orderer_hostname = 'orderer.example.com'
channel_name = 'mychannel'  
joinHyperledger(peer.pod_name,orderer_ip,orderer_hostname,channel_name)

In [ ]:
cmd= "peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json"
executePeerCmd(peer.pod_name,cmd)

B

# Enroll users

In [ ]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray/projects/invoice-hyperledger-fabric/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='upload')
    uploadFromLocal(local_path='/home/jovyan/scray/projects/invoice-hyperledger-fabric/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
#createPeerUser(peers['oly0'],'bretten')     

In [ ]:
peer = peers['oly0']
# build local , upload tool to sharepoint 
#uploadWalletToolToSharepoint()
#installPeerWalletTool(peer)

createPeerUser(peer,'bretten')   

# Install and aprove chaincode

In [ ]:
local_orgs
peers['oly0'].pod_name

In [ ]:
#installAndAproveChaincode(peers['schalke3'])
for name in local_orgs:
    installAndAproveChaincode(peers[name])

In [ ]:
downloadCrtsToPeer(local_orgs, execute_org=local_orgs[0])

In [ ]:
peer lifecycle chaincode commit -o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com 
--tls --cafile $CERT_BASE_PATH/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem --channelID mychannel --name basic --peerAddresses peer0.org1.example.com:7051 --tlsRootCertFiles $CERT_BASE_PATH/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt --peerAddresses peer0.org2.example.com:9051 --tlsRootCertFiles ${CERT_BASE_PATH}/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt --version 1.0 --sequence 1


In [ ]:
#addBashEnvToPeer(peers['schalke3'])
downloadCrtsToPeer(local_orgs)
downloadCrtsToPeer(['schalke3'], execute_org='schalke3')

In [ ]:
def commit():
    try:    
        #crtnew = '\" + getCrt(org='klaus') + '\"
        getCrt(org='klaus')
        output = subprocess.check_output(['peer', 'lifecycle','chaincode','commit','-o',addr0,'--ordererTLSHostnameOverride',orderer,
                                          '--tls','--cafile',ordererpem,'--channelID','mychannel','--name','basic',
                                          '--peerAddresses',addr1, '--tlsRootCertFiles', crt1, 
                                          '--peerAddresses', addr2, '--tlsRootCertFiles', crt2, 
                                          '--peerAddresses', 'peer0.scray.org', '--tlsRootCertFiles', '/tmp/new.crt',
                                          #'--peerAddresses', 'kubernetes.research.dev.seeburger.de:32190', '--tlsRootCertFiles', '/tmp/new.crt',
                                          '--version', '1.0', '--sequence', '1','--tlsRootCertFiles'])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
    
def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    


def getCrtToLocalCmd(org='',topath='/tmp',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    crtname = getCrtName(org, domain)
    return str('curl --user ' + user + ':' + pwd + \
            ' http://' + host + '/ca/' + channel + '/' + crtname +  ' -o ' +  ' /tmp/' + crtname)
        
        
def getCrtToLocal(org='',topath='/tmp',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + getCrtName(org, domain), '-o', '/tmp/' + getCrtName(org, domain)])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)       
    
def getCrtCmdParameters():
    crts=''
    for name in local_orgs:
        if name in peers:
            peer = peers[name] 
            crts = crts + str(' --peerAddresses ' + getPeer0Address(peer)  + ' --tlsRootCertFiles ' +  '/tmp/' + getCrtName(org=peer.name))
    return crts

In [ ]:
#local_orgs, crts
crts = getCrtCmdParameters()

In [ ]:
#addr='peer0.kubernetes.research.dev.seeburger.de'
crts

commit chaincode

In [ ]:
addr0 = 'orderer.example.com:7050'
orderer = 'orderer.example.com'
ordererpem = '/tmp/tlsca.example.com-cert.pem'
cmd1 = 'peer lifecycle chaincode commit -o ' + addr0 + ' --ordererTLSHostnameOverride ' + orderer + ' --tls --cafile ' + ordererpem + ' --channelID mychannel --name basic'
cmd2 = ' --version 1.0 --sequence 1'

def getPart(peer, org='kubernetes.research.dev.seeburger.de'):
    return getCrtToLocalCmd(org=peer.name)
    #return '--peerAddresses ' + org + ':' + peer.listen_port + ' --tlsRootCertFiles ' +  crt1

destpeer = peers[local_orgs[0]]    
for name in local_orgs:
    if name in peers:
        peer = peers[name]    
        cmd = getPart(peer)
        executePeerCmd(destpeer.pod_name,cmd)



In [ ]:
executePeerCmd(destpeer.pod_name,cmd1 + crts + cmd2)

In [ ]:
#cmd1 + crts + cmd2
crts

In [ ]:
peer lifecycle chaincode queryinstalled --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32733 --tlsRootCertFiles /tmp/sally3-kubernetes.research.dev.seeburger.de-ca.crt

In [ ]:
pod_name   = peers['sally3'].pod_name
executePeerCmd(pod_name, "peer lifecycle chaincode queryinstalled --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32733 --tlsRootCertFiles /tmp/sally3-kubernetes.research.dev.seeburger.de-ca.crt --output json")

# test chaincode

In [ ]:
from kubernetesTool import toCmd
from kubernetesTool import executePeerCmd
import json

In [ ]:
#peer chaincode query -C mychannel -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'

def readAsset(peer, asset='asset1'):
    #callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    cmd0 = str('\'' + callopt + "\'")
    try:
        #cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
        cmd1 = 'CORE_PEER_ADDRESS=\"' + getPeer0Address(peer) + '\" && '
        cmd2 = ['peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c']
        cmd = toCmd(cmd2) + ' ' + cmd0
        output = executePeerCmd(peer.pod_name, cmd1 + cmd)
        #print(output)
        if output[0] == '{':
            return json.loads(output)
    except Exception as e:
        print(e)
    
asset = readAsset(peers['oly0'], 'asset7')
asset
#asset['ID'] = 'asset11'

In [ ]:
for key, value in asset.items():
    print(key,value, type(value))
    
    
#import facturxWidget
#widget = facturxWidget.facturx_widgets(asset, '/',hideUnused=True) 


#write asset to json file
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(asset, f, ensure_ascii=False, indent=4)

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('data.p', 'wb') as fp:
    pickle.dump(asset, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def toCmd(strlist):
    return ''.join(str(e + ' ') for e in strlist)[:-1]

#toCmd(asset.values())
#print(e) for e in asset.items()

#toCmd(list(asset.values()))

#["asset11","green","10","Jin Soo","500"]
asset['ID'] = 'asset7'
arglist = '[' + ''.join(('\"' + str(e) + '\",') for e in asset.values())[:-1] + ']' 
print(arglist)

In [ ]:
def createAsset(peer,crts,arglist):
    cmd1 = 'peer chaincode invoke -o orderer.example.com:7050 --tls true --cafile /tmp/tlsca.example.com-cert.pem  -C mychannel -n basic'
    cmd3= '{"function":"CreateAsset","Args":' + arglist + '}'
    cmd2 = "-c "
    cmd = str(cmd1 + ' ' + crts + ' ' + cmd2 + '\'' + cmd3 + '\'')
    #print(cmd)
    return executePeerCmd(peer.pod_name, cmd)
                   
createAsset(peers['oly0'],crts,arglist)
    
#executePeerCmd(peers['oly0'].pod_name,printCmdForPeer(crts))

In [ ]:
peers['oly0'].pod_name

In [ ]:
args = ["asset2","blue","5","Tomoko", "300"]
arglist = ''.join(str('\"' + e + '\",') for e in args)[:-1]
  
callopt= '{\"function\":\"CreateAsset\",\"Args\":[' + arglist + ']}'
#peer chaincode invoke -C mychannel -n basic 
#-c '{"function":"CreateAsset","Args":["asset1","blue","5","Tomoko", "300"]}' 
#--peerAddresses peer0.kubernetes.research.dev.seeburger.de:31866 --tlsRootCertFiles /tmp/hafen0-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30755 --tlsRootCertFiles /tmp/hafen1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30948 --tlsRootCertFiles /tmp/hafen3-kubernetes.research.dev.seeburger.de-ca.crt 
#-o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com --tls --cafile /tmp/tlsca.example.com-cert.pem

            

cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
cmd = ['peer', 'chaincode','invoke', '-C', 'mychannel', '-n', 'basic', '-c',"'" + callopt + "'"]

#certs

In [ ]:
peer chaincode invoke -o orderer.example.com:7050 --tls true --cafile /tmp/tlsca.example.com-cert.pem  -C mychannel -n basic --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30084 --tlsRootCertFiles /tmp/sally1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31521 --tlsRootCertFiles /tmp/sally2-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:32733 --tlsRootCertFiles /tmp/sally3-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31018 --tlsRootCertFiles /tmp/sally4-kubernetes.research.dev.seeburger.de-ca.crt -c '{"function":"CreateAsset","Args":["asset1","green","10", "Jin Soo","500"]}'
                        
peer chaincode query -C mychannel -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'


In [ ]:

addBashEnvToPeer(peer.pod_name, peer.listen_port)

In [ ]:
print(pod_name)
executePeerCmd(pod_name,"hostname")

In [ ]:
 peer chaincode invoke -C mychannel -n basic -c '{"function":"GetAllAssets","Args":[]}' --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31866 --tlsRootCertFiles /tmp/hafen0-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30755 --tlsRootCertFiles /tmp/hafen1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30948 --tlsRootCertFiles /tmp/hafen3-kubernetes.research.dev.seeburger.de-ca.crt -o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com --tls --cafile /tmp/tlsca.example.com-cert.pem


In [ ]:
executePeerCmd(peers['hafen0'].pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"GetAllAssets\",\"Args\":[]}'")

In [ ]:
executePeerCmd(peers['hafen0'].pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset1\"]}'")

In [ ]:
executePeerCmd(pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[]}'")

In [ ]:
executePeerCmd(peers['hafen0'].pod_name,"peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset1\"]}' --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30773 --tlsRootCertFiles /tmp/sally0-kubernetes.research.dev.seeburger.de-ca.crt")

In [ ]:
executePeerCmd(peers['hafen0'].pod_name,"export CORE_PEER_ADDRESS=peer0.kubernetes.research.dev.seeburger.de:31866 && peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset1\"]}'")

In [ ]:
CreateAsset(ctx contractapi.TransactionContextInterface, id, color string, size int, owner string, appraisedValue int)

In [ ]:
peer chaincode invoke -C mychannel -n basic -c '{"function":"CreateAsset","Args":["asset1","blue","5","Tomoko", "300"]}' --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31866 --tlsRootCertFiles /tmp/hafen0-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30755 --tlsRootCertFiles /tmp/hafen1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30948 --tlsRootCertFiles /tmp/hafen3-kubernetes.research.dev.seeburger.de-ca.crt -o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com --tls --cafile /tmp/tlsca.example.com-cert.pem

peer chaincode invoke -C mychannel -n basic -c '{"function":"ReadAsset","Args":["asset1"]}' --peerAddresses peer0.kubernetes.research.dev.seeburger.de:31866 --tlsRootCertFiles /tmp/hafen0-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30755 --tlsRootCertFiles /tmp/hafen1-kubernetes.research.dev.seeburger.de-ca.crt --peerAddresses peer0.kubernetes.research.dev.seeburger.de:30948 --tlsRootCertFiles /tmp/hafen3-kubernetes.research.dev.seeburger.de-ca.crt -o orderer.example.com:7050 --ordererTLSHostnameOverride orderer.example.com --tls --cafile /tmp/tlsca.example.com-cert.pem
peer chaincode query  -C mychannel -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'


In [ ]:
executePeerCmd(pod_name, "peer chaincode query -C mychannel -n basic -c '{\"function\":\"ReadAsset\",\"Args\":[\"asset2\"]}'")

In [ ]:
executePeerCmd(pod_name,  "peer chaincode query -C mychannel -n basic -c '{\"function\":\"GetAllAssets\",\"Args\":[]}'")

In [ ]:
setPeerEnvironment(peername)

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'lara2-5d8bd488d6-kll5n' -c lara2 -- /bin/bash pwd

In [ ]:
########## execute commands in container
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c maja3 -- /bin/sh -c "ls -l / && hostname"
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "ls -l / && hostname"
#!/home/jovyan/work/usr/bin/kubectl  exec -it $POD_NAME -c scray-peer-cli -- /bin/bash -c "echo 'export ORDERER_CA=/tmp/tlsca.example.com-cert.pem' >> /root/.bashrc"

In [ ]:
#setPeerEnvironment('tanja2')
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  /bin/sh export ORDERER_CA="/tmp/tlsca.example.com-cert.pem"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  ORDERER_HOSTNAME="orderer.example.com"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CHANNEL_NAME="mychannel"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32222"
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli --  CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"


!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- env | fgrep ORDERER_HOSTNAME

In [ ]:
peername='maja2'
setPeerEnvironment(peername)
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- env | fgrep ORDERER
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- which export
print(getPod(peername),getPort(peername=peername, name='peer-listen'))

In [ ]:
export ORDERER_CA=/tmp/tlsca.example.com-cert.pem
export ORDERER_HOSTNAME="orderer.example.com"
export CHANNEL_NAME="mychannel"
export CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32190"
export CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"
export SHARED_FS_HOST=10.15.136.41:30080
export SHARED_FS_USER=scray
export SHARED_FS_PW=scray

curl --user $SHARED_FS_USER:$SHARED_FS_PW http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem > /tmp/tlsca.example.com-cert.pem
curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o ./org3_update_in_envelope.pb
peer channel signconfigtx -f org3_update_in_envelope.pb
curl --user scray:scray -T ./org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb
  

In [ ]:
!ls -l ./*.pb

#sign update
#createEnvironmentForPeer(localmspid=mspid2, tlsrootcert=crt2, mspconfigpath=msp2, peeraddress=addr2)
#setPeerEnvironment('carla1')
setPeerEnvironment('tanja2')
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- curl --user scray:scray http://10.15.136.41:30080/upload/org3_update_in_envelope.pb -o ./org3_update_in_envelope.pb
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- peer channel signconfigtx -f org3_update_in_envelope.pb
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- curl --user scray:scray -T ./org3_update_in_envelope.pb http://10.15.136.41:30080/upload/org3_update_in_envelope.pb
  

# Install and aprove chaincode

In [ ]:
setPeerEnvironment('tanja2')
installAndAproveChaincode()

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

In [ ]:
#setPeerEnvironment('klaus')
!echo  $POD_NAME $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

In [ ]:
#!ping orderer.example.com 7050
!more /etc/hosts

# ------------------------------------------------------------------------

In [ ]:
#kubectl get pods -l k8s-app=kube-dns -o jsonpath={.items[*].spec.containers[*].name}
#kubectl get pods hafen1-69947bff88-2pgxs -o json
#cmd= ['get', 'pods', 'hafen1-69947bff88-2pgxs', '-o', 'jsonpath={.items[*].spec.containers[*].name}']
cmd= ['get', 'pods', 'hafen1-69947bff88-2pgxs', '-o', 'json']
#f:spec': {'f:containers': {'k:{"name":"hafen1"}
len(json.loads(executeKubectlCmd(cmd).decode('ascii'))['spec']['containers'])

# Shell only

In [ ]:
#!echo $GOSSIP_PORT, $PEER_NAME
getPort(peername=peername, name='peer-listen')

In [ ]:
GOSSIP_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")
PEER_LISTEN_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
PEER_CHAINCODE_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-chaincode')].nodePort}")


In [ ]:
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-chaincode')].nodePort}")

In [ ]:
os.environ['GOSSIP_PORT'] = "32068"  
os.environ['PEER_LISTEN_PORT'] = "30251"
os.environ['PEER_CHAINCODE_PORT'] = "30052"
!echo $GOSSIP_PORT
#!echo $(/home/jovyan/work/usr/bin/kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-gossip')].nodePort}")

In [ ]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer-$PEER_NAME \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=$PEER_NAME \
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:$PEER_LISTEN_PORT \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:$GOSSIP_PORT \
 --from-literal=CORE_PEER_LOCALMSPID=${PEER_NAME}MSP

In [ ]:
!/home/jovyan/work/usr/bin/kubectl apply -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/$PEER_NAME/k8s-peer.yaml

In [ ]:
POD_NAME=$(/home/jovyan/work/usr/bin/kubectl  get pod -l app=peer0-org1-scray-org -o jsonpath="{.items[0].metadata.name}")
kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
!echo $(/home/jovyan/work/usr/bin/kubectl  get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")


In [ ]:
os.environ['POD_NAME'] = "tom-6647c77bbb-hjlkc"
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME

# Delete old peer

In [ ]:
!/home/jovyan/work/usr/bin/kubectl delete configmap hl-fabric-peer-klaus
!/home/jovyan/work/usr/bin/kubectl delete -f /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/klaus/k8s-peer.yaml
!/home/jovyan/work/usr/bin/kubectl delete service klaus

In [ ]:
#deletePeer('carla1')
deletePeer('ff')

In [ ]:
!/home/jovyan/work/usr/bin/kubectl get configmap 
!/home/jovyan/work/usr/bin/kubectl get deployments 
#!/home/jovyan/work/usr/bin/kubectl get svc -o json
!/home/jovyan/work/usr/bin/kubectl get svc

# Create new peer and 'apply'

In [ ]:
!/home/jovyan/work/usr/bin/kubectl create configmap hl-fabric-peer \
 --from-literal=hostname=kubernetes.research.dev.seeburger.de \
 --from-literal=org_name=OrgSally \ 
 --from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:30003 \
 --from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:30001 \
 --from-literal=CORE_PEER_LOCALMSPID=OrgSallyMSP

!/home/jovyan/work/usr/bin/kubectl apply -f https://raw.githubusercontent.com/scray/scray/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/k8s-peer.yaml        

1. Der neue Peer wurde erzeugt. All relevant infos and cryptos for the application is uploaded to external sharepoint
2. Now the application needs to be signed by other peers

# Now the peer can join the network

In [ ]:
import os
import json
import subprocess
import pandas as pd

!/home/jovyan/work/usr/bin/kubectl get pods
os.environ['POD_NAME'] = 'peer0-org1-scray-org-5c4d69f556-snz5c'

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl get pods | fgrep peer0
#!POD_NAME=$(/home/jovyan/work/usr/bin/kubectl get pod -l app=peer0-org1-scray-org | tail -n 1 | cut -d' ' -f1)
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'mychannel'

!echo $POD_NAME
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
os.environ['ORDERER_IP'] = '10.15.128.97'
os.environ['ORDERER_HOSTNAME'] = 'orderer.example.com'
os.environ['CHANNEL_NAME'] = 'super2'

!echo $POD_NAME
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP $ORDERER_HOSTNAME $CHANNEL_NAME

In [ ]:
peers['olya0'].__dict__

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'olya0-c658f7997-9wh22'  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh 10.15.128.97 'orderer.example.com'  7050 super2 10.14.128.38:30080 

In [ ]:
PEER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=7050
PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty peer0-org1-scray-org-84ddc5757f-glbgn -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# Install and aprove chaincode

In [ ]:
# Host where the chaincode is running
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK

# QR Code

In [ ]:
#!pip install qrcode

In [ ]:
#Import Library
import qrcode#Generate QR Code
img=qrcode.make('Hello World')
img.save('hello.png')

In [ ]:
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data("https://scray.org")
qr.make(fit=True)
img = qr.make_image(fill_color="red", back_color="black")
img.save("medium.png")

In [ ]:
! pip install opencv-python

In [ ]:
img=cv2.imread("medium.png")
det=cv2.QRCodeDetector()
val, pts, st_code=det.detectAndDecode(img)
print(val)

# TMP

In [ ]:
print(getPod(peername),getPort(peername=peername, name='peer-listen'))

In [ ]:
export ORDERER_CA="/tmp/tlsca.example.com-cert.pem"
export ORDERER_HOSTNAME="orderer.example.com"
export CHANNEL_NAME="mychannel"
export CORE_PEER_ADDRESS="peer0.kubernetes.research.dev.seeburger.de:32190"
export CORE_PEER_MSPCONFIGPATH="/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/users/Admin@kubernetes.research.dev.seeburger.de/msp"

peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json
   

In [ ]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- peer

In [ ]:
#pip install PyYAML
import yaml